## Úkol 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shippy/czechitas-ai-data/blob/main/notebooks/assignment-03.ipynb)


### Nastavte si API klíč

1. Navštivte [OpenAI API Dashboard](https://platform.openai.com/api-keys) a vytvořte si API klíč.
2. V levé liště klikněte na tlačítko "Secrets" a přidejte předtím vytvořený API klíč do proměnné `GOOGLE_API_KEY` - buď "napřímo" přes "+ Add new secret", nebo přes "Gemini API keys" > "Import key from Google AI Studio".

(Pokud jste otevřeli tento notebook lokálně, např. ve VSCode, můžete si API klíč nastavit v souboru `.env` - viz README k tomuto repozitáři.)

### Setup a test

Spusťte následující dvě buňky. Pokud vám AI řekne, jak se má, vše funguje správně.

In [1]:
try:
    from google.colab import userdata
    _secret = userdata.get("OPENAI_API_KEY")
    %pip install instructor openai python-dotenv rich
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("OPENAI_API_KEY")

In [ ]:
import instructor
from openai import OpenAI
import os
from rich import print

try:
    if not _secret:
        raise ValueError("API klíč nebyl nastaven!")
except NameError:
    print("Nastavte si API klíč v proměnné GOOGLE_API_KEY a znovu spusťte *celý* notebook včetně předchozí buňky")

_client = OpenAI()
client = instructor.from_openai(_client)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you? Respond with an emotion."},
    ],
    model="gpt-4o",
    response_model=str,
)
print(test)


### Úkol, část 1

Vytáhněte ze nestrukturovaných odpovědí níže definované datové struktury.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

Occupation = Literal["gardener", "teacher", "doctor", "engineer", "lawyer", "scientist", "artist", "writer", "pensioner", "other"]

class Citizen(BaseModel):
    name: str = Field(..., description="Full name of the citizen")
    age: int
    occupation: Occupation
    interests: list[str]

In [8]:
census_responses = [
    "Jsem osmadvacetiletý Honza Pálka, pracuju v Semilhotce jako čistič kanálů. Rád hraju šachy a na bicí.",
    "Jsem Jana Nováková, je mi 30, dojíždím do Semilhotské školy učit. Ráda čtu knihy a jezdím na hory.",
    "Jsem čtyřicetiletý Petr Válek, pracuju v Semilhotce jako zedník. Rád cestuji a hraju fotbal.",
    "Jsem padesátiletý Vladimír Dvořák, pracuju v Semilhotce jako policista. Rád cestuji a hraju kulečník.",
    "Jsem šedesátiletý Jaroslav Bílek, pracuju v Semilhotce jako zahradník. Rád cestuji a hraju kulečník.",
    "Jsem sedmdesátiletý Karel Dvořák, pracuju v Semilhotce jako zedník. Rád cestuji a hraju kulečník.",
    "Jsem osmdesátiletý Josef Novák, pracuju v Semilhotce jako inženýr. Rád cestuji a hraju kulečník.",
    "Jsem devadesátiletý Igor Hnízdo, jsem tu penzista. Rád cestuji a hraju kulečník.",
]


- Tip 1: můžete použít i prompt engineering (tj. první message s `"role": "system"`), aby AI lépe poznala, co chcete.
- Tip 2: `client.chat.completions.create` umí vracet i `response_model=Citizen` nebo i `response_model=list[Citizen]`

Dávalo by smysl na základě dat přidefinovat do třídy `Citizen` nějaký atribut, abyste zachytili více informací?

In [ ]:
# Zde řešte úkol 1. (Můžete si vytvořit i více buněk!)

### Úkol, část 2

Vygenerujte dataset nebo sadu datasetů, který popíše fiktivní městečko Semilhotka. Začněte např. s datovou strukturou `Citizen` následovně:

In [ ]:

# A vygenerujeme nějaké obyvatele
citizens = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Generate 10 citizens of Semilhotka."},
    ],
    model="gpt-4o",
    response_model=list[Citizen],
)
print(citizens)

Ideálně bychom chtěli, aby výčet kulminoval objektem Semilhotka:

```python
from pydantic import BaseModel

class Semilhotka(BaseModel):
    citizens: list[Citizen]
    # a další atributy

my_semilhotka = Semilhotka(
    citizens=citizens,
    # a další proměnné, které jste si vygenerovali
)
```

In [ ]:
# Zde řešte úkol 2. (Můžete si vytvořit i více buněk!)

## Export dat

Pro jednoduchost si vaši vesnici exportujeme jako Python pickle soubor. Ten si potom z levého panelu můžete stáhnout a použít v jiném notebooku (jste-li  v Colabu).

In [ ]:
from pickle import dump

try:
    if my_semilhotka:
        with open("semilhotka.pkl", "wb") as f:
            dump(my_semilhotka, f)
except NameError:
    raise ValueError("Musíte si nejdříve nadefinovat Semilhotku a přiřadit ji do proménně!")
